In [1]:
import csv
#Transform to UTF-8

data_path = './data/user_tag_query.10W.TRAIN' 
csvfile = open(data_path + '-1w.csv', 'w')
writer = csv.writer(csvfile)
writer.writerow(['ID', 'age', 'Gender', 'Education', 'QueryList'])

with open(data_path, 'r',encoding='gb18030',errors='ignore') as f:
    lines = f.readlines()
    for line in lines[0:10000]:
        try:
            line.strip()          
            data = line.split("\t")
            writedata = [data[0], data[1], data[2], data[3]]
            querystr = ''
            data[-1]=data[-1][:-1]
            for d in data[4:]:
                try:
                    cur_str = d.encode('utf8')
                    cur_str = cur_str.decode('utf8')
                    querystr += cur_str + '\t'
                except:
                    continue
                    #print (data[0][0:10])
            querystr = querystr[:-1]
            writedata.append(querystr)
            writer.writerow(writedata)
        except:
            #print (data[0][0:20])
            continue

In [2]:
data_path = './data/user_tag_query.10W.TEST'

csvfile = open(data_path + '-1w.csv', 'w')
writer = csv.writer(csvfile)
writer.writerow(['ID', 'QueryList'])
with open(data_path, 'r',encoding='gb18030',errors='ignore') as f:
    lines = f.readlines()
    for line in lines[0:10000]:
        try:
            data = line.split("\t")
            writedata = [data[0]]
            querystr = ''
            data[-1]=data[-1][:-1]
            for d in data[1:]:
                try:                  
                    cur_str = d.encode('utf8')
                    cur_str = cur_str.decode('utf8')
                    querystr += cur_str + '\t'               
                except:
                    #print (data[0][0:10])
                    continue
            querystr = querystr[:-1]
            writedata.append(querystr)
            writer.writerow(writedata)
        except:
            #print (data[0][0:20])
            continue

In [3]:
import pandas as pd

#Divide into train and test data
trainname = './data/user_tag_query.10W.TRAIN-1w.csv'
testname = './data/user_tag_query.10W.TEST-1w.csv'

data = pd.read_csv(trainname,encoding='gbk')
print (data.info())

data.age.to_csv("./data/train_age.csv", index=False)
data.Gender.to_csv("./data/train_gender.csv", index=False)
data.Education.to_csv("./data/train_education.csv", index=False)
data.QueryList.to_csv("./data/train_querylist.csv", index=False)

data = pd.read_csv(testname,encoding='gbk')
print (data.info())

data.QueryList.to_csv("./data/test_querylist.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9988 entries, 0 to 9987
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         9988 non-null   object
 1   age        9988 non-null   int64 
 2   Gender     9988 non-null   int64 
 3   Education  9988 non-null   int64 
 4   QueryList  9988 non-null   object
dtypes: int64(3), object(2)
memory usage: 390.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9971 entries, 0 to 9970
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         9971 non-null   object
 1   QueryList  9971 non-null   object
dtypes: object(2)
memory usage: 155.9+ KB
None


In [7]:
import pandas as pd
import jieba.analyse
import time
import jieba
import jieba.posseg
import os, sys
#text segmentation

def input(trainname):
    traindata = []
    with open(trainname, 'rb') as f:
        line = f.readline()
        count = 0
        while line:
            try:
                traindata.append(line)
                count += 1
            except:
                print ("error:", line, count)
            line=f.readline()
    return traindata
#start = time.clock()
start = time.perf_counter()

filepath = './data/test_querylist.csv'
QueryList = input(filepath)

writepath = './data/test_querylist_writefile-1w.csv'
csvfile = open(writepath, 'w')

POS = {}
for i in range(len(QueryList)):
    #print (i)
    if i%2000 == 0 and i >=1000:
        print (i,'finished') 
    s = []
    str = ""
    words = jieba.posseg.cut(QueryList[i])
    allowPOS = ['n','v','j']
    for word, flag in words:
        POS[flag]=POS.get(flag,0)+1
        if (flag[0] in allowPOS) and len(word)>=2:
            str += word + " "
            
    cur_str = str.encode('utf8')
    cur_str = cur_str.decode('utf8')
    s.append(cur_str)
    
    csvfile.write(" ".join(s)+'\n')
csvfile.close()

#end = time.clock()
end = time.perf_counter()
print ("total time: %f s" % (end - start))

2000 finished
4000 finished
6000 finished
8000 finished
total time: 305.067053 s


In [14]:
from gensim.models import word2vec
train_path = './data/train_querylist_writefile-1w.csv'
with open(train_path, 'r') as f:
    My_list = []
    lines = f.readlines()
    for line in lines:
        cur_list = []
        line = line.strip()
        data = line.split(" ")
        for d in data:
            cur_list.append(d)
        My_list.append(cur_list)
    
    model = word2vec.Word2Vec(My_list, vector_size=300, window=10,workers=4)  
    savepath = '1w_word2vec_' + '300'+'.model' 

    model.save(savepath)

In [130]:
model.wv.most_similar("清华")

[('特长生', 0.9320876002311707),
 ('中国人民大学', 0.8937985897064209),
 ('北大', 0.8924200534820557),
 ('北京大学', 0.891645073890686),
 ('复旦大学', 0.8911810517311096),
 ('院校', 0.8872029185295105),
 ('清华大学', 0.8815717101097107),
 ('录取线', 0.878574788570404),
 ('南开大学', 0.878221869468689),
 ('金融学', 0.8778314590454102)]

In [52]:
import numpy as np
from gensim import models
#average vector
file_name = './data/train_querylist_writefile-1w.csv'
cur_model = models.Word2Vec.load('1w_word2vec_300.model')
with open(file_name, 'r') as f:
    cur_index = 0
    lines = f.readlines()
    doc_cev = np.zeros((len(lines),300))
    for line in lines:
        word_vec = np.zeros((1,300))
        words = line.strip().split(' ')
        wrod_num = 0
        for word in words:
            #print(words)
            if word in cur_model.wv:
                wrod_num += 1
                word_vec += np.array([cur_model.wv[word]])
        doc_cev[cur_index] = word_vec / float(wrod_num)
        cur_index += 1

In [128]:
import numpy as np
import wordcloud
import imageio
#word cloud
file_name = './data/train_querylist_writefile-1w.csv'

font = 'Dengl.ttf'

words = ''

genderlabel = np.loadtxt(open('./data/train_gender.csv', 'r'), skiprows=1).astype(int)
educationlabel = np.loadtxt(open('./data/train_education.csv', 'r'), skiprows=1).astype(int)

label = genderlabel
index = 2
filename = 'gender2' + 'cloud.png'
bg_pic =  imageio.imread('gender2.png')
#bg_pic =  imageio.imread('female.png')

with open(file_name, 'r') as f:
    cur_index = 0
    lines = f.readlines()
    for line in lines:
        word_vec = np.zeros((1,300))
        
        if(label[cur_index] == index):
            words = words + line
        cur_index += 1  
        
        if cur_index  == 2:
            print(line)
        if cur_index > 100:
            break
 
#print(words)        
c = wordcloud.WordCloud(mask = bg_pic, font_path = font, background_color='white', max_words = 100, repeat = False)
c.generate(words)
c.to_image()
c.to_file(filename)

广州 厨宝 烤箱 世情 人情 花易落 风干 泪痕 厦门 酒店用品 批发市场 不想 支付 原谅 无情 处女座 代表 花朵 鸡胸肉 做法 烤箱 忘记 想起 走到 联塑 排水管 规格 大王 性格 文静 意思 牛奶 奶粉 化蝶 芳草 意思 学会 懂得 学会 福睿斯 斗鱼 厨宝 厨宝 烤箱 禹州 城市 广场 电影院 大王 棕树 图片 棕树 图片 发酵 不想 支付 原谅 无情 城市 下雨 有没有 牛奶 奶粉 做法 烤箱 全脂奶粉 比例 牛肉 厨宝牌 猴配猴 婚姻 可惜 爱情 公寓 不住 笑话 太岁 意思 禹州 城市 广场 电影院 禹州 城市 广场 电影院 烤箱 牌子 全脂奶粉 铁人 面粉 蛋白质 含量 活成 样子 鸡排 做法 放爱 斗鱼 直播 糖浆 鸡胸肉 沙枣 轮胎 气压 轮胎 气压 厨宝 牌子 冰皮 月饼 糯米粉 出去 永远都是 生肖 配对 处女座 配对 处女座 相信 世界 冒险 作为 意思 虎牙 直播 联系 想像 联系 瓦一 小时 鸡排 做法 好吃 卡农 故事 开可 不用 道歉 首歌 婚礼 足够 婚礼 厨宝 人生 秋风 悲画 耳朵 图片 空间 视频 厨宝 官网 走到 生肖 配对 高筋 面粉 杨之华 林肯 处女座 男生 特点 感情 紫罗兰 学会 懂得 王棕 龟头 小红点 烤鸡肉 做法 烤箱 邓超 香菇 鸡片 做法 瓦一 小时 瓦一 小时 黄油 阿里巴巴 批发网 和面机 茉香 绿茶 泡法 意大利 烟肉 厨宝 意思 人生路 冰皮 月饼 做法 配方 分手 厨宝 烤箱 价格表 处女座 男生 恋爱 特点 鸡胸肉 做法 全家 牛至 香菇 鸡块 做法 电池 联塑 南瓜 饺子 做法 蝴蝶 兰花 心寒 意思 人生 人生 拖拉机 图片 月饼 比例 鸡胸肉 做法 婚礼 足够 人民币 生肖 代表 轮胎 气压 处女座 男生 恋爱 注意 铁人 面粉 烤鸡肉 做法 触宝 官网 瑞斯 雅格 打喷嚏 吉凶 沙棕 面包 做法 可惜 女人 出轨 女人 烤肉 做法 确定 爱情 公寓 可惜 没有 红豆相思 诗句 月饼 豆沙 做法 土司面包 做法 人生 联系 想像 爱情 摆在 全脂奶粉 罗勒 图片 罗勒 种子 胡子 原因 走过 懂得 烤鱼 烤箱 温度 时间 厨宝牌 烤箱 分手 鸡胸肉 做法 猴子 头像 简笔画 朋友圈 视频 厨宝 烤箱 牌子 相亲 意大利 烟肉 珍珠奶茶 南瓜 做法 出去 永远都是 手扶拖拉机 猴子 

In [26]:
doc_cev.shape

(9988, 300)

In [27]:
doc_cev[5]

array([ 0.28605239,  0.08456552, -0.03387368,  0.40967785, -0.06074544,
       -0.19303788,  0.29931122,  0.65367988, -0.35404672,  0.34058462,
        0.11514796, -0.29682864,  0.83069158,  0.16118781, -0.21476016,
        0.29299607, -0.06404278, -0.20244132,  0.21500196, -0.32999783,
       -0.58554488,  0.23107728,  0.15186575, -0.30322209, -0.17741637,
        0.04875937, -1.13948402,  0.21164155, -0.01895785, -0.34594541,
        0.25041685,  0.2073958 , -0.05585247, -0.1117806 , -0.22181896,
        0.6193757 ,  0.13302078, -0.03312266, -0.35758141, -0.02361543,
       -0.12798117,  0.14999292, -0.14540141,  0.15569988,  0.28706231,
       -0.18069979, -0.04731872, -0.02267059, -0.01743772,  0.37264246,
        0.06796043,  0.19068663,  0.06682884, -0.14007489,  0.32452205,
        0.21952693,  0.52570341,  0.2173711 ,  0.28917367,  0.28604051,
       -0.32838078,  0.37875476, -0.39212239,  0.09687852, -0.17460263,
       -0.03972751,  0.11020986, -0.29446618, -0.17828588,  0.12

In [31]:
genderlabel = np.loadtxt(open('./data/train_gender.csv', 'r'), skiprows=1).astype(int)
genderlabel.shape

(9988,)

In [32]:
educationlabel = np.loadtxt(open('./data/train_education.csv', 'r'), skiprows=1).astype(int)
educationlabel.shape

(9988,)

In [34]:
agelabel = np.loadtxt(open('./data/train_age.csv', 'r'), skiprows=1).astype(int)
agelabel.shape

(9988,)

In [35]:
def removezero(x, y):
        nozero = np.nonzero(y)
        y = y[nozero]
        x = np.array(x)
        x = x[nozero]
        return x, y
gender_train, genderlabel = removezero(doc_cev, genderlabel)
age_train, agelabel = removezero(doc_cev, agelabel)
education_train, educationlabel = removezero(doc_cev, educationlabel)
print (gender_train.shape,genderlabel.shape)
print (age_train.shape,agelabel.shape)
print (education_train.shape,educationlabel.shape)

(9756, 300) (9756,)
(9815, 300) (9815,)
(9064, 300) (9064,)


In [36]:
import matplotlib.pyplot as plt
import itertools
def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [39]:
import numpy as np
file_name = './data/test_querylist_writefile-1w.csv'
cur_model = models.Word2Vec.load('1w_word2vec_300.model')
with open(file_name, 'r') as f:
    cur_index = 0
    lines = f.readlines()
    doc_cev = np.zeros((len(lines),300))
    for line in lines:
        word_vec = np.zeros((1,300))
        words = line.strip().split(' ')
        wrod_num = 0
        for word in words:
            if word in cur_model.wv:
                wrod_num += 1
                word_vec += np.array([cur_model.wv[word]])
        doc_cev[cur_index] = word_vec / float(wrod_num)
        cur_index += 1

C:\Users\LK867\AppData\Local\Temp\ipykernel_65460\1435954172.py:17: RuntimeWarning: invalid value encountered in true_divide
  doc_cev[cur_index] = word_vec / float(wrod_num)


In [40]:
doc_cev.shape

(9972, 300)

In [41]:
doc_cev[6]

array([-5.43363786e-02,  4.36613412e-01,  2.84264537e-02, -2.04844711e-02,
       -2.35153031e-01, -4.78497279e-01,  2.39278287e-01,  3.87633498e-01,
        2.10046680e-02,  1.30885600e-02,  1.76059343e-01, -9.83039028e-02,
       -2.13489247e-01,  3.03166624e-01, -1.94200236e-01, -3.96401041e-01,
        2.25793197e-01, -8.23055541e-02,  1.33312987e-01,  4.41146614e-02,
        7.42870123e-03,  7.89558207e-02,  5.75791957e-01, -4.57384490e-02,
        3.01354918e-01,  2.56641772e-02, -1.30405654e-01, -2.20521983e-01,
       -2.35177718e-01, -3.69375733e-01,  2.97681757e-02, -2.94269163e-01,
       -9.16692546e-02, -7.02136037e-02, -8.12211835e-02,  2.20430628e-01,
        1.64288955e-01, -5.49386606e-01,  4.37120331e-02, -1.77976631e-01,
        1.76180659e-01,  1.07522886e-02, -3.22695774e-02, -5.22001086e-01,
       -5.28454662e-02,  3.23612767e-01, -1.64170810e-02, -7.11515662e-03,
        4.80080634e-02, -2.60805020e-01, -2.85780312e-01, -8.99825666e-02,
       -2.10516920e-01,  